In [1]:
import keras

Using TensorFlow backend.
/Users/dwyer/opt/miniconda3/envs/cascade/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/dwyer/opt/miniconda3/envs/cascade/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/dwyer/opt/miniconda3/envs/cascade/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/U

I needed to install `cctk`, and switch some import statements in the `nfp` subfolder.

Downgrade `hdf5` from 3.0 to less than 3.0

In [2]:
keras.__version__

'2.2.5'

In [3]:
import tensorflow
import tensorflow.keras

In [4]:
tensorflow.keras.__version__

'2.1.6-tf'

In [5]:
import pandas as pd
import numpy as np
from rdkit import Chem
from nfp.preprocessing import MolAPreprocessor, GraphSequence

import keras
import keras.backend as K

from keras.callbacks import ModelCheckpoint, CSVLogger, LearningRateScheduler

from keras.layers import (Input, Embedding, Dense, BatchNormalization,
                                 Concatenate, Multiply, Add)

from keras.models import Model, load_model

from nfp.layers import (MessageLayer, GRUStep, Squeeze, EdgeNetwork,
                               ReduceBondToPro, ReduceBondToAtom, GatherAtomToBond, ReduceAtomToPro)
from nfp.models import GraphModel
from cascade.apply import predict_NMR_C_xyz,predict_NMR_H_xyz

/Users/dwyer/opt/miniconda3/envs/cascade/lib/python3.6/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.preprocessing.data module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.preprocessing. Anything that cannot be imported from sklearn.preprocessing is now part of the private API.
  warnings.warn(message, FutureWarning)


In [6]:
import os

In [7]:
modelpath_C = os.path.join('cascade', 'trained_model', 'best_model.hdf5')
modelpath_H = os.path.join('cascade', 'trained_model', 'best_model_H_DFTNN.hdf5')

batch_size = 32
atom_means = pd.Series(np.array([0,0,97.74193,0,0,0,0,0,0,0]).astype(np.float64), name='shift')
NMR_model_C = load_model(modelpath_C, custom_objects={'GraphModel': GraphModel,
                                             'ReduceAtomToPro': ReduceAtomToPro,
                                             'Squeeze': Squeeze,
                                             'GatherAtomToBond': GatherAtomToBond,
                                             'ReduceBondToAtom': ReduceBondToAtom})
NMR_model_H = load_model(modelpath_H, custom_objects={'GraphModel': GraphModel,
                                             'ReduceAtomToPro': ReduceAtomToPro,
                                             'Squeeze': Squeeze,
                                             'GatherAtomToBond': GatherAtomToBond,
                                             'ReduceBondToAtom': ReduceBondToAtom})
# NMR_model_C.summary()
# NMR_model_H.summary()

/Users/dwyer/opt/miniconda3/envs/cascade/lib/python3.6/site-packages/tensorflow/python/ops/gradients_impl.py:112: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


In [8]:
ls

Predictor/             nfp/                   test.py*
cascade/               predict_xyzfile.ipynb  uwsgi_params*
db.sqlite3*            static/                xyzfiles/
media/                 test.csv


In [9]:
modelpath_C

'cascade/trained_model/best_model.hdf5'

In [10]:
#Loading DATA/
import glob
files = glob.glob('xyzfiles/*.xyz')
files

['xyzfiles/mol_3.xyz',
 'xyzfiles/mol_2.xyz',
 'xyzfiles/mol_10.xyz',
 'xyzfiles/mol_1.xyz',
 'xyzfiles/mol_5.xyz',
 'xyzfiles/mol_4.xyz',
 'xyzfiles/mol_6.xyz',
 'xyzfiles/mol_7.xyz',
 'xyzfiles/mol_9.xyz',
 'xyzfiles/mol_8.xyz']

In [11]:
# C predicting NMR
pred_data_C = pd.DataFrame()
for i, file in enumerate(files[1:2]):
    try:
        mols, spreadShift = predict_NMR_C_xyz(file, NMR_model_C)
        pred_data_C = pd.concat([pred_data_C,spreadShift])
    except:
        pass

/Users/dwyer/opt/miniconda3/envs/cascade/lib/python3.6/site-packages/openbabel/__init__.py:14: UserWarning: "import openbabel" is deprecated, instead use "from openbabel import openbabel"
  warnings.warn('"import openbabel" is deprecated, instead use "from openbabel import openbabel"')
1it [00:00, 256.16it/s]


In [12]:
pred_data_C

,atom_index,mol_id,predicted
0,1,xyzfiles/mol_2.xyz,12.23
1,2,xyzfiles/mol_2.xyz,153.96
2,4,xyzfiles/mol_2.xyz,144.70
3,5,xyzfiles/mol_2.xyz,113.13
4,6,xyzfiles/mol_2.xyz,122.84
5,7,xyzfiles/mol_2.xyz,162.47
6,10,xyzfiles/mol_2.xyz,137.83
7,11,xyzfiles/mol_2.xyz,123.56
8,12,xyzfiles/mol_2.xyz,130.04
9,13,xyzfiles/mol_2.xyz,126.87


In [13]:
# H predicitions
pred_data_H = pd.DataFrame()
for i, file in enumerate(files[1:2]):
    try:
        mols, spreadShift = predict_NMR_H_xyz(file, NMR_model_H)
        pred_data_H = pd.concat([pred_data_H, spreadShift])
    except:
        pass

/Users/dwyer/opt/miniconda3/envs/cascade/lib/python3.6/site-packages/openbabel/__init__.py:14: UserWarning: "import openbabel" is deprecated, instead use "from openbabel import openbabel"
  warnings.warn('"import openbabel" is deprecated, instead use "from openbabel import openbabel"')
1it [00:00, 492.40it/s]


In [14]:
pred_data_C

,atom_index,mol_id,predicted
0,1,xyzfiles/mol_2.xyz,12.23
1,2,xyzfiles/mol_2.xyz,153.96
2,4,xyzfiles/mol_2.xyz,144.70
3,5,xyzfiles/mol_2.xyz,113.13
4,6,xyzfiles/mol_2.xyz,122.84
5,7,xyzfiles/mol_2.xyz,162.47
6,10,xyzfiles/mol_2.xyz,137.83
7,11,xyzfiles/mol_2.xyz,123.56
8,12,xyzfiles/mol_2.xyz,130.04
9,13,xyzfiles/mol_2.xyz,126.87


In [15]:
pred_data_H

,atom_index,mol_id,predicted
0,16,xyzfiles/mol_2.xyz,2.47
1,17,xyzfiles/mol_2.xyz,2.46
2,18,xyzfiles/mol_2.xyz,2.63
3,19,xyzfiles/mol_2.xyz,7.38
4,20,xyzfiles/mol_2.xyz,6.87
5,21,xyzfiles/mol_2.xyz,6.50
6,22,xyzfiles/mol_2.xyz,6.99
7,23,xyzfiles/mol_2.xyz,7.31
8,24,xyzfiles/mol_2.xyz,7.04
9,25,xyzfiles/mol_2.xyz,7.32
